<a href="https://colab.research.google.com/github/KiranH1007/-MLB-fan-engagement-project/blob/handling-of-cloud-function-and-deployment-of-recommendation-function/Live%20Game%20Sentiment%20Analysis/ContentRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Required Libraries
!pip install google-cloud-storage google-cloud-bigquery nltk pandas

In [ ]:
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025' # Make sure this is the correct bucket name
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly without getting the bucket metadata first
# This assumes the bucket is public and blob_name includes the full path
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file
raw_data = blob.download_as_text()

# Print the data (optional)
#print(raw_data[0:100])

In [ ]:
# load json with pandas
# Data set is 2025-mlb-fan-favs-follows.json
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df_fanfav = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df_fanfav = pd.DataFrame(json_data)

# Preview data
print(df_fanfav.head())

print(df_fanfav.info())

In [ ]:
# load json with pandas
# data set altogether with  multiple json files
# Note ******** currently doing with 5 files later do the Scalable
import pandas as pd
import json
from google.cloud import storage
import re

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
file_pattern = 'datasets/mlb-fan-content-interaction-data/'

# List to store dataframes from each file
all_dfs = []

blobs = storage_client.list_blobs(bucket_name, prefix=file_pattern)
# for blob in blobs:
#     print(blob.name)

filtered_blobs = [blob for blob in blobs if re.match(r'datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-\d+\.json$', blob.name)]

# limiting the dataset
filtered_blobs = filtered_blobs[:5]

for blob_name in filtered_blobs:

       print(f"Processing file: {blob_name.name}")
       # Get the blob directly
       blob = storage_client.bucket(bucket_name).blob(blob_name.name)

       # Download the file as bytes
       data = blob.download_as_bytes()

       # Decode the bytes to a string
       raw_data = data.decode('utf-8')

       # Load the data using json.loads
       try:
           json_data = json.loads(raw_data)
           df = pd.DataFrame([json_data])
       except json.JSONDecodeError:
           json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
           df = pd.DataFrame(json_data)

       #print(df.head())
       # Data Cleaning: Remove Redundancy (similar to your example)
       df.drop(columns=['content_headline'], inplace=True, errors='ignore')  # Ignore if column doesn't exist
       df['content_id'] = df['content_type'] + ':' + df['slug'].str.lower()
       df.drop(columns=['slug', 'content_type'], inplace=True, errors='ignore')
       df.drop(columns=['date_time_date'], inplace=True, errors='ignore')

       # Add the cleaned dataframe to the list
       all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.head()

In [ ]:
# Analysis of DataFrames are named df_fanfav, final_df
common_columns = list(set(df_fanfav.columns) & set(final_df.columns))
print("Common Columns:", common_columns)


In [ ]:
#without cleanup and duplicate columns

import pandas as pd

# it should be outer for all unique data and inner for common data

# Merge the result with the third dataset which has content type headline
merged_df = pd.merge(df_fanfav, final_df, on=common_columns, how='outer')

print(merged_df.count())

merged_df.head()

In [ ]:
import pickle
with open('merged_df.pkl', 'wb') as f:
   pickle.dump(merged_df, f)

checkpoint saved for merged_df dataset


In [ ]:
with open('merged_df.pkl', 'rb') as f:
   merged_df = pickle.load(f)

In [ ]:
user_row = merged_df[merged_df['user_id'] == 'some_user_id']
print(user_row)

Fan Engagement Recommendation System using only in context with user id who followed  followed_player_ids column and player_tags based on content id column.
1. Overview

This system provides personalized content recommendations to fans based on their followed players and popular content. It leverages a dataset called merged_df, which contains information about users, their followed players, and content with associated player tags.

2. Data Structure

The system relies on the following data structures:

user_player_mapping: A dictionary that maps each user ID to a set of player IDs they follow. This mapping is derived from the followed_player_ids column in merged_df.
player_content_mapping: A dictionary that maps each player tag to a set of content IDs associated with that tag. This mapping is created using the player_tags and content_id columns in merged_df.
3. Recommendation Logic

The core recommendation logic is implemented in the recommend_content function:

Input: The function takes a user_id and top_n (number of recommendations) as input.
Player-Based Recommendations: If the user_id is found in user_player_mapping, it iterates through the followed players and retrieves associated content from player_content_mapping. These recommendations are added to a set called recommendations.
Fallback: If no recommendations are found based on followed players (e.g., for new users), a fallback strategy is used. In this case, it recommends the most popular content from merged_df based on the content_id column.
Output: The function returns a list of up to top_n recommended content IDs.
4. Example Usage


user_recommendations = recommend_content('some_user_id', top_n=5)
for i in user_recommendations:
    print(i)
Use code with caution
This code snippet demonstrates how to get recommendations for a specific user and print them.



In [ ]:
# first sample takes more time
# import pandas as pd
# from collections import defaultdict

# # 1. Build User-Player and Player-Content Mappings
# user_player_mapping = defaultdict(set)
# player_content_mapping = defaultdict(set)

# for _, row in merged_df.iterrows():
#     user_id = row['user_id']
#     followed_players = row['followed_player_ids']
#     content_id = row['content_id']
#     player_tags = row['player_tags']

#     if isinstance(followed_players, str):
#         for player in followed_players.split(','):
#             user_player_mapping[user_id].add(player.strip())

#     if isinstance(player_tags, str):
#         for tag in player_tags.split(','):
#             player_content_mapping[tag.strip()].add(content_id)

# print(user_player_mapping)
# print("hi")
# print(player_content_mapping)


In [ ]:
# Convert columns to string type, handling NaN values
merged_df['followed_player_ids'] = merged_df['followed_player_ids'].astype(str)
merged_df['player_tags'] = merged_df['player_tags'].astype(str)

In [ ]:
# optimised way
import pandas as pd
from collections import defaultdict

# 1. Build User-Player Mapping
# Explode followed_player_ids and apply a lambda function
user_player_df = merged_df.assign(followed_player_ids=merged_df['followed_player_ids'].str.split(',')).explode('followed_player_ids')
user_player_df['followed_player_ids'] = user_player_df['followed_player_ids'].str.strip()
user_player_mapping1 = user_player_df.groupby('user_id')['followed_player_ids'].apply(set).to_dict()


# 2. Build Player-Content Mapping
# Explode player_tags and apply a lambda function
player_content_df = merged_df.assign(player_tags=merged_df['player_tags'].str.split(',')).explode('player_tags')
player_content_df['player_tags'] = player_content_df['player_tags'].str.strip()
player_content_mapping1 = player_content_df.groupby('player_tags')['content_id'].apply(set).to_dict()


In [ ]:
import random

# Get a list of user IDs from the mapping
user_ids = list(user_player_mapping1.keys())

# Randomly select 5 user IDs
sample_user_ids = random.sample(user_ids, 35)

# Print the mappings for the selected user IDs M1RJCXEE8IY1NU5
for user_id in sample_user_ids:
  print(f"User ID: {user_id}, Followed Players: {user_player_mapping1[user_id]}")

In [ ]:
import random

# Get a list of user IDs from the mapping
player_ids = list(player_content_mapping1.keys())

for player_id in player_ids:
  print(f"player tags {player_id}, contentid: {player_content_mapping1[player_id]}")

In [ ]:
# 2. Recommendation Function
def recommend_content(user_id, top_n=10):
    recommendations = set()
    if user_id in user_player_mapping1:
        for player in user_player_mapping1[user_id]:
            if player in player_content_mapping1:
                recommendations.update(player_content_mapping1[player])
                print(player_content_mapping1[player])

    # Fallback (if no recommendations based on followed players)
    if not recommendations:
        print("no recommendation")
        # Recommend most popular content or other strategies
        popular_content = merged_df['content_id'].value_counts().index.tolist()
        recommendations.update(popular_content[:top_n])

    return list(recommendations)[:top_n]

# Example Usage:
# user_recommendations = recommend_content('some_user_id', top_n=5)
# for i in user_recommendations:
#   print(i)

In [ ]:
# Use pickle to save these data structures to files. This makes them accessible to the Cloud Function later.
import pickle

with open('user_player_mapping1.pkl', 'wb') as f:
    pickle.dump(user_player_mapping1, f)

with open('player_content_mapping1.pkl', 'wb') as f:
    pickle.dump(player_content_mapping1, f)

popular_content = merged_df['content_id'].value_counts().index.tolist()
with open('popular_content.pkl', 'wb') as f:
    pickle.dump(popular_content, f)

### `Cloud Function`

---



# Google cloud authentication and installation

In [ ]:
!apt-get update && apt-get install -y google-cloud-sdk

In [ ]:
!gcloud auth login

In [ ]:
# set the project id to fanengagementapp
!gcloud config set project mlbfanengagementapp

# Main.py which is function in google cloud deployment

add requirements.txt and all pkl files and main.py

In [ ]:
# location of pkl files
# !ls -lh user_player_mapping1.pkl
# !ls -lh player_content_mapping1.pkl
# !ls -lh popular_content.pkl


In [ ]:
# Trial 1

# import pickle

# def recommend_content_cloud_function(request):
#     """HTTP Cloud Function to handle recommendation requests."""

#     # Load data from local files (will be packaged with the function)
#     with open('user_player_mapping1.pkl', 'rb') as f:
#         user_player_mapping1 = pickle.load(f)
#     with open('player_content_mapping1.pkl', 'rb') as f:
#         player_content_mapping1 = pickle.load(f)
#     with open('popular_content.pkl', 'rb') as f:
#         popular_content = pickle.load(f)

#     # Get user_id from request
#     request_json = request.get_json(silent=True)
#     user_id = request_json.get('user_id', '')

#     if not user_id:
#         return {'error': 'Missing user_id in request'}, 400

#     try:
#         recommendations = set()
#         if user_id in user_player_mapping1:
#             for player in user_player_mapping1[user_id]:
#                 if player in player_content_mapping1:
#                     recommendations.update(player_content_mapping1[player])

#         # Fallback
#         if not recommendations:
#             recommendations.update(popular_content)

#         return {'recommendations': list(recommendations)[:10]}, 200

#     except Exception as e:
#         return {'error': str(e)}, 500

In [ ]:
# The Crucial Fix: Proper Logging and Error Handling: for cloud function optimization
%%writefile main.py
import pickle
import logging
import os 

def recommend_content_cloud_function(request):
    """HTTP Cloud Function to handle recommendation requests."""

    try:
        # Load data from local files (will be packaged with the function)

        print("Loading data from all pkl files")

        files_to_check = ['user_player_mapping1.pkl', 'player_content_mapping1.pkl', 'popular_content.pkl']
        all_files_present = all(os.path.isfile(file) for file in files_to_check)

        print("all pkl files are present and ready")

        if all_files_present:
            try:
                with open('user_player_mapping1.pkl', 'rb') as f:
                    user_player_mapping1 = pickle.load(f)
                with open('player_content_mapping1.pkl', 'rb') as f:
                    player_content_mapping1 = pickle.load(f)
                with open('popular_content.pkl', 'rb') as f:
                    popular_content = pickle.load(f)
                
                print("Files loaded successfully")
            except Exception as e:
                print(f"An error occurred while loading files: {e}")
        else:
            print("One or more files are missing.")

        if request.method == 'GET':  # Handle GET request
            user_id = request.args.get('userid')
            print(f"Received GET request for user_id: {user_id}") # Log the user_id

        elif request.method == 'POST':  # Handle POST request (if needed)
            request_json = request.get_json(silent=True)
            user_id = request_json.get('user_id', '')
            print(f"Received POST request for user_id: {user_id}") # Log the user_id

        else:
            logging.warning(f"Received unsupported method: {request.method}") # Log the request method
            return 'Method Not Allowed', 405  # Return 405 for other methods

        if not user_id:
            logging.error("Missing user_id in request") # Log the missing user_id error
            return {'error': 'Missing user_id in request'}, 400

        recommendations = set()
        if user_id in user_player_mapping1:
            for player in user_player_mapping1[user_id]:
                if player in player_content_mapping1:
                    recommendations.update(player_content_mapping1[player])

        # Fallback
        if not recommendations:
            recommendations.update(popular_content)

        result = {'recommendations': list(recommendations)[:10]}
        print(f"Recommendations for user {user_id}: {result}") # Log the recommendations
        return result, 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")  # Log the full exception with stack trace
        return {'error': f"Internal Server Error: {e}"}, 500  # Return a 500 error

# # Dummy request for testing
# class DummyRequest:
#     def __init__(self, args=None, json=None, method='GET'):
#         self.args = args if args is not None else {}
#         self._json = json if json is not None else {}
#         self.method = method

#     def get_json(self, silent=False):
#         return self._json


# request_get = DummyRequest(args={'userid': '7P45I0SCJ3AAIUW'})
# response_get = recommend_content_cloud_function(request_get)

# if response_get[1] == 200:
#     print("Success:", response_get[0])
# else:
#     print("Error:", response_get)

In [ ]:
# all colab pip packages are there in requirements currently not required for so many packages in cloud
#pip freeze > requirements.txt

For Profiling that is memory and speed of execution

In [ ]:
#!pip install line_profiler  # Install the line profiler

In [ ]:
# #Improvised code with profiling and also testing with dummy user
# %%writefile main.py
# import time
# import pickle
# import logging
# import psutil

# # Configure logging (do this at the beginning of your function)
# logging.basicConfig(level=logging.ERROR)

# def recommend_content_cloud_function(request):
#     """HTTP Cloud Function to handle recommendation requests."""
#     start_time = time.time()  # Start timing the entire function

#     try:
#         # Load data from local files (will be packaged with the function)
#         load_start = time.time()
#         with open('user_player_mapping1.pkl', 'rb') as f:
#             user_player_mapping1 = pickle.load(f)
#         with open('player_content_mapping1.pkl', 'rb') as f:
#             player_content_mapping1 = pickle.load(f)
#         with open('popular_content.pkl', 'rb') as f:
#             popular_content = pickle.load(f)
#         load_end = time.time()
#         print(f"File loading time: {load_end - load_start} seconds")

#         if request.method == 'GET':  # Handle GET request
#             user_id = request.args.get('userid')
#             print(f"Received GET request for user_id: {user_id}")  # Log the user_id

#         elif request.method == 'POST':  # Handle POST request (if needed)
#             request_json = request.get_json(silent=True)
#             user_id = request_json.get('user_id', '')
#             print(f"Received POST request for user_id: {user_id}")  # Log the user_id

#         else:
#             logging.warning(f"Received unsupported method: {request.method}")  # Log the request method
#             return 'Method Not Allowed', 405  # Return 405 for other methods

#         if not user_id:
#             logging.error("Missing user_id in request")  # Log the missing user_id error
#             return {'error': 'Missing user_id in request'}, 400

#         recommendations = set()
#         recommendation_start = time.time()
#         if user_id in user_player_mapping1:
#             for player in user_player_mapping1[user_id]:
#                 if player in player_content_mapping1:
#                     recommendations.update(player_content_mapping1[player])

#         # Fallback
#         if not recommendations:
#             recommendations.update(popular_content)
#         recommendation_end = time.time()
#         print(f"Recommendation logic time: {recommendation_end - recommendation_start} seconds")

#         result = {'recommendations': list(recommendations)[:10]}
#         print(f"Recommendations for user {user_id}: {result}")  # Log the recommendations
#         end_time = time.time()  # End timing the entire function
#         print(f"Total function execution time: {end_time - start_time} seconds")
#         return result, 200

#     except Exception as e:
#         logging.exception(f"An error occurred: {e}")  # Log the full exception with stack trace
#         return {'error': f"Internal Server Error: {e}"}, 500  # Return a 500 error


# # Dummy request for testing
# class DummyRequest:
#     def __init__(self, args=None, json=None, method='GET'):
#         self.args = args if args is not None else {}
#         self._json = json if json is not None else {}
#         self.method = method

#     def get_json(self, silent=False):
#         return self._json


# request_get = DummyRequest(args={'userid': '7P45I0SCJ3AAIUW'})
# response_get = recommend_content_cloud_function(request_get)

# if response_get[1] == 200:
#     print("Success:", response_get[0])
# else:
#     print("Error:", response_get)

# Deployment of function to google cloud

gcloud functions deploy recommend_content_cloud_function \
    --runtime python39 \
    --trigger-http \
    --source your-cloud-function-directory \
    --entry-point recommend_content_cloud_function \
    --memory 512M  # Or your chosen memory
    --timeout 60s  # Set timeout to 60 seconds
    --allow-unauthenticated # set for public url usecase

In [ ]:
!gcloud functions deploy recommend_content_cloud_function --runtime python39 --trigger-http --source /content/ --entry-point recommend_content_cloud_function --memory 512M 